<h1>Segmenting and Clustering Neighborhoods in Toronto: Part 2</h1>

Prepare the data frame based on what has been done in part 1

In [24]:
import pandas as pd # library for data analsysis
# define the dataframe columns
column_names = ['Postcode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

from bs4 import BeautifulSoup
from urllib.request import urlopen

soup = BeautifulSoup(urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read(), "html.parser")

for row in soup.select("table[class='wikitable sortable'] tr"):
    if len(row.select("td:nth-of-type(1)")) == 0:
        continue
    if len(row.select("td:nth-of-type(2)")) == 0:
        continue
    if len(row.select("td:nth-of-type(3)")) == 0:
        continue
    postcode = row.select("td:nth-of-type(1)")[0].string
    borough = row.select("td:nth-of-type(2)")[0].string
    if borough is None:
        borough = row.select("td:nth-of-type(2) a")[0].string
    neighborhood = row.select("td:nth-of-type(3)")[0].string
    if neighborhood is None:
        neighborhood = row.select("td:nth-of-type(3) a")[0].string
    
    neighborhoods = neighborhoods.append({'Postcode': postcode.strip(),
                                          'Borough': borough.strip(),
                                          'Neighborhood': neighborhood.strip()}, ignore_index=True)

neighborhoods = neighborhoods[neighborhoods['Borough'] != 'Not assigned']
neighborhoodsGroupByPostcode = neighborhoods.groupby(['Postcode', 'Borough'])
neighborhoods = neighborhoodsGroupByPostcode['Neighborhood'].apply(lambda x: ', '.join(x)).to_frame().reset_index()
neighborhoods.loc[neighborhoods['Neighborhood'] == 'Not assigned', 'Neighborhood'] = neighborhoods['Borough']
neighborhoods.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Let's download the geospatial data and save it as a CSV file called **geospatial_data.csv**

In [2]:
!wget -q -O 'geospatial_data.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


Now load the data into a data frame

In [4]:
geospatial_data_df = pd.read_csv('geospatial_data.csv')
geospatial_data_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now let's join the two data frames to get what we expect

In [25]:
neighborhoods = neighborhoods.set_index('Postcode').join(geospatial_data_df.set_index('Postal Code'))
neighborhoods = neighborhoods.reset_index()
neighborhoods.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
